# Text-To-Text LLM Server

**important: Select venv Python Interpreter before you start**

This repository is designed to be used with Visual Studio Code and Docker DevContainer.

![dev-container](../img/dev-container.png)

## 1. Setup

**Instructions:**

a) Download model

```bash
huggingface-cli download bartowski/Mistral-7B-Instruct-v0.3-exl2 \
    --revision 1a09a351a5fb5a356102bfca2d26507cdab11111 \
    --local-dir ~/.gai/models/exllamav2-mistral7b \
    --local-dir-use-symlinks False
```

or

```bash
huggingface-cli download bartowski/dolphin-2.9-llama3-8b-exl2 \
    --revision 6521bd8b0f793a038f85d445316c94cdd0957d8e \
    --branch 4_25 \
    --local-dir ~/.gai/models/exllamav2-mistral7b \
    --local-dir-use-symlinks False
```

b) Create gai.yml in ~/.gai

```yaml
generators:
    ttt:
        default: "ttt-exllamav2-dolphin"
        configs:
            ttt-exllamav2-dolphin:
                type: "ttt"
                engine: "exllamav2"
                model: "dolphin"
                name: "ttt-exllamav2-dolphin"
                model_path: "models/exllamav2-dolphin"
                model_basename: "model"
                max_seq_len: 8192
                prompt_format: "mistral"
                hyperparameters:
                    temperature: 0.85
                    top_p: 0.8
                    top_k: 50
                    max_tokens: 1000
                    tool_choice: "auto"
                    max_retries: 5
                    stop: ["<|im_end|>", "</s>", "[/INST]"]
                extra:                    
                    no_flash_attn: true
                    seed: null
                    decode_special_tokens: false
                module:
                    name: "gai.ttt.server.gai_exllamav2"
                    class: "GaiExLlamav2"
            ttt-exllamav2-mistral7b:
                type: "ttt"
                engine: "exllamav2"
                model: "mistral7b"
                name: "ttt-exllamav2-mistral7b"
                model_path: "models/exllamav2-mistral7b"
                model_basename: "model"
                max_seq_len: 8192
                prompt_format: "mistral"
                hyperparameters:
                    temperature: 0.85
                    top_p: 0.8
                    top_k: 50
                    max_tokens: 1000
                    tool_choice: "auto"
                    max_retries: 5
                    stop: ["<|im_end|>", "</s>", "[/INST]"]
                extra:
                    no_flash_attn: true
                    seed: null
                    decode_special_tokens: false
                module:
                    name: "gai.ttt.server.gai_exllamav2"
                    class: "GaiExLlamav2"
```

---

## 2. Smoke Test

In [1]:
# check .gairc
import os
gairc=None
with open(os.path.expanduser("~/.gairc"),"r") as f:
    gairc = f.read()
print(gairc)

# check ~/.gairc (if docker created .gairc)
import json
jsoned=json.loads(gairc)
assert os.path.expanduser(jsoned["app_dir"])=="/home/kakkoii1337/.gai"

# check ~/.gai (if docker created the mount point)
assert os.path.exists(os.path.expanduser("~/.gai"))

# Initiate
from gai.lib.server.singleton_host import SingletonHost
from gai.lib.common.utils import free_mem
from rich.console import Console
console=Console()

from gai.ttt.server.config.pydantic.ttt_config import TTTConfig
ttt_config = TTTConfig(
    type="ttt",
    engine="exllamav2",
    model="dolphin",
    name="ttt-exllamav2-dolphin",
    model_path="models/exllamav2-dolphin",
    max_seq_len=8192,
    prompt_format="mistral",
    hyperparameters={
        "temperature": 0.85,
        "top_p": 0.8,
        "top_k": 50,
        "max_tokens": 1000,
        "tool_choice": "auto",
        "max_retries": 5,
        "stop": ["<|im_end|>", "</s>", "[/INST]"],
    },
    extra={
        "no_flash_attn": True,
        "seed": None,
        "decode_special_tokens": False        
    },
    module={
        "name": "gai.ttt.server.gai_exllamav2",
        "class": "GaiExLlamav2"
    }
)

# before loading
free_mem()
try:
    with SingletonHost.GetInstanceFromConfig(ttt_config) as host:

        # after loading
        free_mem()
except Exception as e:
    raise e
finally:
    # after disposal
    free_mem()

{"app_dir":"/home/kakkoii1337/.gai"}



Free memory: 6.05 GB

                      Performance load_config                      
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 4.91 seconds │ 4.91 seconds │
│ CPU Memory        │          - MB │     21.16 MB │     21.16 MB │
│ CUDA Memory       │       6.04 GB │     -0.03 GB │      6.07 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

                      Performance load_model                       
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 0.03 seconds │ 0.03 seconds │
│ CPU Memory        │          - MB │      0.22 MB │      0.22 MB │
│ CUDA Memory       │       6.07 GB │      0.00 GB │      6.06 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

                       Performance load_cache                        
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃  Change Value ┃   Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 22.48 seconds │ 22.48 seconds │
│ CPU Memory        │          - MB │      24.20 MB │      24.20 MB │
│ CUDA Memory       │       6.06 GB │       4.71 GB │       1.35 GB │
└───────────────────┴───────────────┴───────────────┴───────────────┘

                    Performance load_tokenizer                     
┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric            ┃ Initial Value ┃ Change Value ┃  Final Value ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ Function Duration │             - │ 2.69 seconds │ 2.69 seconds │
│ CPU Memory        │          - MB │     49.66 MB │     49.66 MB │
│ CUDA Memory       │       1.35 GB │      0.01 GB │      1.34 GB │
└───────────────────┴───────────────┴──────────────┴──────────────┘

Free memory: 1.34 GB

Free memory: 5.85 GB

---

## 3. Integration Test

### Startup

In [2]:
host = SingletonHost.GetInstanceFromConfig(ttt_config, verbose=False)
host.load()
generator = host.generator
free_mem()

Free memory: 1.32 GB

1.3221702575683594

### a) Test streaming

In [3]:
response = host.generator.create(
    messages=[{"role":"user","content":"Tell me a one paragraph story"},
                {"role":"assistant","content":""}],
    stream=True)
for message in response:
    if message.choices[0].delta.content:
        print(message.choices[0].delta.content, end="", flush=True)
   

A young boy named Timmy found a magical coin while playing in the park. As soon as he picked it up, he noticed that it had the power to grant wishes. Overwhelmed with excitement, he wished for a giant ice cream sundae. To his delight, a huge sundae appeared in front of him. He took a bite and it was the most delicious thing he had ever tasted. He continued to make wishes, each one more extravagant than the last, until the coin ran out of magic. As he sat there, a sad realization dawned on him: with all his wishes fulfilled, he was left with nothing but an empty sundae bowl and a longing for more.

### b) Test generation

In [4]:

response = host.generator.create(
    messages=[{"role":"user","content":"Tell me a one paragraph story"},
                {"role":"assistant","content":""}],
    stream=False)
print(response.choices[0].message.content)


A young boy named Tim, who lived in a small village, found a strange old key one day while playing near the river. Intrigued, he decided to use it to explore an abandoned castle on the hill. Inside, he discovered a hidden room filled with treasure and an ancient scroll. The scroll revealed that Tim was the lost heir to the castle and its riches. With this newfound knowledge, Tim used his wealth to help his village prosper, turning it into a thriving community. His life became a tale of adventure, wisdom, and generosity, inspiring everyone who heard his story.


### c) Test Tool Calling

In [5]:
messages = [
    {"role":"user","content":"What is the current time in Singapore?"},
    {"role":"assistant","content":""}
]
tool_choice="required"
tools = [
    {
        "type": "function",
        "function": {
            "name": "google",
            "description": "The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_query": {
                        "type": "string",
                        "description": "The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively."
                    }
                },
                "required": ["search_query"]
            }
        }
    }
]
response = host.generator.create(
    messages=messages,
    tools=tools,
    tool_choice=tool_choice,
    stream=False)
print(response)


ChatCompletion(id='chatcmpl-fb1da187-d8fe-4549-98ba-15bf9d92cab9', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_b25141a3-fbe5-41dd-b98a-da2f4c0b02f8', function=Function(arguments='{"search_query": "current time in Singapore"}', name='google'), type='function')]))], created=1734485527, model='exllamav2-mistral7b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=50, prompt_tokens=418, total_tokens=468, completion_tokens_details=None, prompt_tokens_details=None))


### d) Test Structured Output

In [6]:
# Define Schema
from pydantic import BaseModel
class Book(BaseModel):
    title: str
    summary: str
    author: str
    published_year: int

text = """Foundation is a science fiction novel by American writer
Isaac Asimov. It is the first published in his Foundation Trilogy (later
expanded into the Foundation series). Foundation is a cycle of five
interrelated short stories, first published as a single book by Gnome Press
in 1951. Collectively they tell the early story of the Foundation,
an institute founded by psychohistorian Hari Seldon to preserve the best
of galactic civilization after the collapse of the Galactic Empire.
"""
response = host.generator.create(messages=[{'role':'user','content':text},{'role':'assistant','content':''}], 
    json_schema=Book.schema(),
    stream=False
    )
print(response)


/tmp/ipykernel_4556/2228101676.py:18: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json_schema=Book.schema(),


ChatCompletion(id='chatcmpl-12d9675f-d4fe-4af8-84f9-9f4d76e2a5bd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n   "title": "Foundation",\n   "summary": "Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.",\n   "author": "Isaac Asimov",\n   "published_year": 1951\n }', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1734485536, model='exllamav2-mistral7b', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_toke

### Teardown

In [7]:
del host.generator.model
del host.generator.cache
del host.generator.tokenizer
del host.generator
import gc,torch
gc.collect()
torch.cuda.empty_cache()
free_mem()

Free memory: 1.53 GB

1.5302543640136719

---

## 4. API Test

**Instructions**:

a) Open Debug Icon and select **Python Debugger: gai-ttt server (dolphin)**

b) Press `F5` to start the API server.

c) Wait for the server to start.


**Tests**:

Run the following cells to test the API.

### a) Test Generating

In [1]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Tell me a story.\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"max_tokens\":100,\
        \"tool_choice\": \"none\"}"
        

{"id":"chatcmpl-b35983a8-e65f-4379-89a8-ed1fa7711711","choices":[{"finish_reason":"length","index":0,"logprobs":null,"message":{"content":"Once upon a time, in a small village nestled between the mountains and the sea, there lived a young woman named Amara. She was known throughout the village for her exceptional talent as a healer. Her knowledge of herbs and natural remedies was unparalleled, and she was often sought out by neighboring villages for her wisdom.\n\n One day, a traveler from a distant land arrived in the village. He was a tall man with a long, flowing beard,","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1734485610,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":100,"prompt_tokens":21,"total_tokens":121,"completion_tokens_details":null,"prompt_tokens_details":null}}

### b) Test Streaming

In [2]:
import json
import httpx
import asyncio
from openai import ChatCompletion

json_payload = {
    "temperature": 0.2,
    "max_tokens": 50,
    "stream": "true",  # This should probably be a boolean True, not "true"
    "messages": [
        {
            "role": "user",
            "content": "Tell me a one paragraph story."
        },
        {
            "role": "assistant",
            "content": ""
        }
    ]
}
async def http_post_async(json_payload):

    # Send the POST request using httpx with streaming
    async with httpx.AsyncClient(timeout=30.0) as client:
        async with client.stream("POST", "http://localhost:12031/gen/v1/chat/completions", json=json_payload) as response:
            response.raise_for_status()
            async for chunk in response.aiter_text():  # Use aiter_text() to handle decoding
                chunk=json.loads(chunk)
                chunk=chunk["choices"][0]["delta"]["content"]
                if chunk:  # Check for non-empty chunks
                    print(chunk, end="", flush=True)

response=await http_post_async(json_payload)


A young boy named Timmy found a magical coin while playing in the park. He rubbed it and wished for a pet dragon, and suddenly, a baby dragon appeared before him. They became the best of friends, going on many adventures together, and Tim

### c) Test Tool Calling

In [3]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"What is the current time in Singapore\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"tools\": [\
            {\
                \"type\": \"function\",\
                \"function\": {\
                    \"name\": \"google\",\
                    \"description\": \"The 'google' function is a powerful tool that allows the AI to gather external information from the internet using Google search. It can be invoked when the AI needs to answer a question or provide information that requires up-to-date, comprehensive, and diverse sources which are not inherently known by the AI. For instance, it can be used to find current date, current news, weather updates, latest sports scores, trending topics, specific facts, or even the current date and time. The usage of this tool should be considered when the user's query implies or explicitly requests recent or wide-ranging data, or when the AI's inherent knowledge base may not have the required or most current information. The 'search_query' parameter should be a concise and accurate representation of the information needed.\",\
                    \"parameters\": {\
                        \"type\": \"object\",\
                        \"properties\": {\
                            \"search_query\": {\
                                \"type\": \"string\",\
                                \"description\": \"The search query to search google with. For example, to find the current date or time, use 'current date' or 'current time' respectively.\"\
                            }\
                        },\
                        \"required\": [\"search_query\"]\
                    }\
                }\
            }\
        ],\
        \"tool_choice\": \"required\"}"

{"id":"chatcmpl-e5d82c26-bd0d-4ba0-bb03-4776abd4e500","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":[{"id":"call_ea605d64-5210-4620-bdba-4194e8ceeb51","function":{"arguments":"{\"search_query\": \"current time in Singapore\"}","name":"google"},"type":"function"}]}}],"created":1734485622,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":44,"prompt_tokens":417,"total_tokens":461,"completion_tokens_details":null,"prompt_tokens_details":null}}

### d) Test JSON Schema

In [4]:
%%bash
curl -X POST \
    http://localhost:12031/gen/v1/chat/completions \
    -H 'Content-Type: application/json' \
    -s \
    -N \
    -d "{\"model\":\"exllamav2-mistral7b\", \
        \"messages\": [ \
            {\"role\": \"user\",\"content\": \"Foundation is a science fiction novel by American writer \
            Isaac Asimov. It is the first published in his Foundation Trilogy (later \
            expanded into the Foundation series). Foundation is a cycle of five \
            interrelated short stories, first published as a single book by Gnome Press \
            in 1951. Collectively they tell the early story of the Foundation, \
            an institute founded by psychohistorian Hari Seldon to preserve the best \
            of galactic civilization after the collapse of the Galactic Empire.\"}, \
            {\"role\": \"assistant\",\"content\": \"\"} \
        ],\
        \"json_schema\": {\"properties\": \
            {\"title\": \
                {\"title\": \"Title\", \"type\": \"string\"}, \
                    \"summary\": {\"title\": \"Summary\", \"type\": \"string\"}, \
                    \"author\": {\"title\": \"Author\", \
                    \"type\": \"string\"\
                }, \
                \"published_year\": {\
                    \"title\": \"Published Year\", \
                    \"type\": \"integer\"}}, \
                \"required\": [\
                    \"title\", \
                    \"summary\", \
                    \"author\", \
                    \"published_year\"\
                ], \
                \"title\": \"Book\", \
                \"type\": \"object\"\
            },\
        \"stream\": false }"

{"id":"chatcmpl-159656fb-f60c-4aa8-8134-3c688fa0b6e5","choices":[{"finish_reason":"stop","index":0,"logprobs":null,"message":{"content":"{\n   \"title\": \"Foundation\",\n   \"summary\": \"Foundation is a science fiction novel by American writer Isaac Asimov. It is the first published in his Foundation Trilogy (later expanded into the Foundation series). Foundation is a cycle of five interrelated short stories, first published as a single book by Gnome Press in 1951. Collectively they tell the early story of the Foundation, an institute founded by psychohistorian Hari Seldon to preserve the best of galactic civilization after the collapse of the Galactic Empire.\",\n   \"author\": \"Isaac Asimov\",\n   \"published_year\": 1951\n }","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":null}}],"created":1734485630,"model":"exllamav2-mistral7b","object":"chat.completion","service_tier":null,"system_fingerprint":null,"usage":{"completion_tokens":148,"prompt_toke

### e) Shut down the API Service

---

## 5. Docker

This test should **NOT** be run in devcontainer.

**Instructions:** 

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: up**

**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **docker-compose: down**

#### Smoke Test

In [1]:
%%bash
curl http://localhost:12031


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    35  100    35    0     0    964      0 --:--:-- --:--:-- --:--:--  1060


{"message":"gai-ttt-svr-exllamav2"}

**Tests:**

Repeat the API test (#)

**Tear Down:**

- Press **CTRL+SHIFT+P** > **Tasks: Run Task** > **Docker: stop**

### Debugging

a) Container must be started with "python -m debugpy --listen 0.0.0.0:5678 main.py"

b) Port 5678 must be opened.

c) Click on "Debug" in Tool bar

d) Select "Attach" > "Run and Debug"

e) Add a "breakpoint" in the code

f) Run the API test to see if it trigger the breakpoint.